In [13]:
# Importing the required libraries
import requests
import json
import pandas as pd
import time
import datetime
import os
import numpy as np

# Universalis API no key required
# Setting the API Key for XIVAPI from the environment variable XIVAPI_KEY
xivapi_key = os.environ.get('XIVAPI_KEY')

# Setting the URL for the Universalis API
# https://docs.universalis.app/
univeralis_url = 'https://universalis.app/api/'

# Setting the URL for the XIVAPI
# https://xivapi.com/docs
xivapi_url = 'https://xivapi.com/'

In [14]:
tinctures = pd.read_csv('..\\data\\tinctures.csv')
tinctures

,ID,Icon,Name,Url,UrlType,_,_Score
0,39727,/i/020000/020710.png,Grade 8 Tincture of Strength,/Item/39727,Item,item,0
1,39728,/i/020000/020709.png,Grade 8 Tincture of Dexterity,/Item/39728,Item,item,0
2,39729,/i/020000/020707.png,Grade 8 Tincture of Vitality,/Item/39729,Item,item,0
3,39730,/i/020000/020706.png,Grade 8 Tincture of Intelligence,/Item/39730,Item,item,0
4,39731,/i/020000/020708.png,Grade 8 Tincture of Mind,/Item/39731,Item,item,0


In [15]:
# now to get the data for all the marketable consumables
# Initialize an empty df to store all results
tinc_results = pd.DataFrame()

# Initialize the Data center to Aether
# Initialize the histInSeconds to 1 years

world = "Aether"
years = 4
histInSeconds = years*365*24*60*60 # year in seconds

# Loop through all the marketable consumables
# Each request will be for a single consumable
# We have 2321 marketable consumables and we have to limit the requests to 25 per second
# We will use a sleep timer to limit the requests to 25 per second
failed_requests = []

for index, row in tinctures.iterrows():
    # Make the request
    itemID = row['ID']
    response = requests.get(f'https://universalis.app/api/v2/history/{world}/{itemID}?entriesToReturn=999999&entriesWithin={histInSeconds}&minSalePrice=0&maxSalePrice=2147483647')
    # check if the request was successful
    if response.status_code != 200:
        print(f"Request failed for {itemID}")
        # store the itemID in a list to retry later
        failed_requests.append(itemID)
        continue
    response = response.json()
    # normalize the data
    response = pd.json_normalize(response)
    # concatenate the data to the all_results df
    tinc_results = pd.concat([tinc_results, response,], ignore_index=True)
    time.sleep(0.04)


In [16]:
tinc_results

,itemID,lastUploadTime,entries,dcName,regularSaleVelocity,nqSaleVelocity,hqSaleVelocity,stackSizeHistogram.1,stackSizeHistogram.2,stackSizeHistogram.3,...,stackSizeHistogramNQ.43,stackSizeHistogramNQ.52,stackSizeHistogramNQ.53,stackSizeHistogramNQ.56,stackSizeHistogramNQ.73,stackSizeHistogramNQ.22,stackSizeHistogramNQ.44,stackSizeHistogramNQ.47,stackSizeHistogramNQ.62,stackSizeHistogramNQ.83
0,39727,1710956243472,"[{'hq': True, 'pricePerUnit': 2700, 'quantity'...",Aether,32271.000000,78.142860,32192.857000,201,135,1717,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,39728,1710956464436,"[{'hq': True, 'pricePerUnit': 2862, 'quantity'...",Aether,13222.714000,20.571428,13202.143000,101,94,919,...,1.0,1.0,1.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN
2,39729,1710951391617,"[{'hq': True, 'pricePerUnit': 2974, 'quantity'...",Aether,24.714285,0.000000,24.714285,16,7,74,...,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,39730,1710956305589,"[{'hq': True, 'pricePerUnit': 2823, 'quantity'...",Aether,10675.857000,48.714287,10627.143000,62,92,784,...,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN
4,39731,1710956329840,"[{'hq': True, 'pricePerUnit': 2837, 'quantity'...",Aether,13636.714000,21.571428,13615.143000,81,103,934,...,1.0,NaN,2.0,NaN,NaN,2.0,1.0,3.0,1.0,2.0


In [17]:
# using the tinctures df to add the name of the item to the results
tinc_results = tinc_results.merge(tinctures, how='left', left_on='itemID', right_on='ID')
tinc_results = tinc_results.drop(columns=['ID'])
tinc_results

,itemID,lastUploadTime,entries,dcName,regularSaleVelocity,nqSaleVelocity,hqSaleVelocity,stackSizeHistogram.1,stackSizeHistogram.2,stackSizeHistogram.3,...,stackSizeHistogramNQ.44,stackSizeHistogramNQ.47,stackSizeHistogramNQ.62,stackSizeHistogramNQ.83,Icon,Name,Url,UrlType,_,_Score
0,39727,1710956243472,"[{'hq': True, 'pricePerUnit': 2700, 'quantity'...",Aether,32271.000000,78.142860,32192.857000,201,135,1717,...,NaN,NaN,NaN,NaN,/i/020000/020710.png,Grade 8 Tincture of Strength,/Item/39727,Item,item,0
1,39728,1710956464436,"[{'hq': True, 'pricePerUnit': 2862, 'quantity'...",Aether,13222.714000,20.571428,13202.143000,101,94,919,...,NaN,NaN,NaN,NaN,/i/020000/020709.png,Grade 8 Tincture of Dexterity,/Item/39728,Item,item,0
2,39729,1710951391617,"[{'hq': True, 'pricePerUnit': 2974, 'quantity'...",Aether,24.714285,0.000000,24.714285,16,7,74,...,NaN,NaN,NaN,NaN,/i/020000/020707.png,Grade 8 Tincture of Vitality,/Item/39729,Item,item,0
3,39730,1710956305589,"[{'hq': True, 'pricePerUnit': 2823, 'quantity'...",Aether,10675.857000,48.714287,10627.143000,62,92,784,...,NaN,NaN,NaN,NaN,/i/020000/020706.png,Grade 8 Tincture of Intelligence,/Item/39730,Item,item,0
4,39731,1710956329840,"[{'hq': True, 'pricePerUnit': 2837, 'quantity'...",Aether,13636.714000,21.571428,13615.143000,81,103,934,...,1.0,3.0,1.0,2.0,/i/020000/020708.png,Grade 8 Tincture of Mind,/Item/39731,Item,item,0


In [18]:
strength_history = pd.json_normalize(tinc_results['entries'][tinc_results['Name'].str.contains("Strength")].values[0])
# add the name and itemID of the item to the df
strength_history['Name'] = tinc_results['Name'][tinc_results['Name'].str.contains("Strength")].values[0]
strength_history['itemID'] = tinc_results['itemID'][tinc_results['Name'].str.contains("Strength")].values[0]
#convert the timestamp to a date
strength_history['timestamp'] = pd.to_datetime(strength_history['timestamp'], unit='s')
# save the data to a csv
strength_history.to_csv('..\\data\\strength_tincture_market.csv', index=False)
strength_history


,hq,pricePerUnit,quantity,buyerName,onMannequin,timestamp,worldName,worldID,Name,itemID
0,True,2700,99,Bib Kek,False,2024-03-20 17:11:23,Gilgamesh,63,Grade 8 Tincture of Strength,39727
1,True,2700,99,Mythrogar Adarva,False,2024-03-20 17:09:19,Gilgamesh,63,Grade 8 Tincture of Strength,39727
2,True,2699,50,Mythrogar Adarva,False,2024-03-20 17:09:12,Gilgamesh,63,Grade 8 Tincture of Strength,39727
3,True,2699,50,Mythrogar Adarva,False,2024-03-20 17:09:10,Gilgamesh,63,Grade 8 Tincture of Strength,39727
4,True,2699,50,Mythrogar Adarva,False,2024-03-20 17:09:08,Gilgamesh,63,Grade 8 Tincture of Strength,39727
...,...,...,...,...,...,...,...,...,...,...
173281,True,25000,10,Gwyn Graves,False,2023-05-23 13:53:16,Gilgamesh,63,Grade 8 Tincture of Strength,39727
173282,True,25000,50,Gwyn Graves,False,2023-05-23 13:53:15,Gilgamesh,63,Grade 8 Tincture of Strength,39727
173283,False,6969,3,Xeno Izanagi,False,2023-05-23 12:39:47,Gilgamesh,63,Grade 8 Tincture of Strength,39727
173284,True,19997,15,Sonya Windwalker,False,2023-05-23 11:50:44,Gilgamesh,63,Grade 8 Tincture of Strength,39727


In [19]:
vitality_history = pd.json_normalize(tinc_results['entries'][tinc_results['Name'].str.contains("Vitality")].values[0])
# add the name and itemID of the item to the df
vitality_history['Name'] = tinc_results['Name'][tinc_results['Name'].str.contains("Vitality")].values[0]
vitality_history['itemID'] = tinc_results['itemID'][tinc_results['Name'].str.contains("Vitality")].values[0]
#convert the timestamp to a date
vitality_history['timestamp'] = pd.to_datetime(vitality_history['timestamp'], unit='s')
# save the data to a csv
vitality_history.to_csv('..\\data\\Vitality_tincture_market.csv', index=False)
vitality_history

,hq,pricePerUnit,quantity,buyerName,onMannequin,timestamp,worldName,worldID,Name,itemID
0,True,2974,20,Reina Uberall,False,2024-03-17 23:06:14,Gilgamesh,63,Grade 8 Tincture of Vitality,39729
1,True,1850,30,Elsha Linnette,False,2024-03-14 12:58:43,Midgardsormr,65,Grade 8 Tincture of Vitality,39729
2,True,2996,24,Samara Hana,False,2024-03-14 12:17:15,Adamantoise,73,Grade 8 Tincture of Vitality,39729
3,True,2997,99,Samara Hana,False,2024-03-14 12:17:14,Adamantoise,73,Grade 8 Tincture of Vitality,39729
4,True,2000,33,Sair Hyskaris,False,2024-03-13 16:39:09,Gilgamesh,63,Grade 8 Tincture of Vitality,39729
...,...,...,...,...,...,...,...,...,...,...
790,True,25000,3,Jatoro Brathgar,False,2023-05-24 21:35:07,Jenova,40,Grade 8 Tincture of Vitality,39729
791,True,6000,3,Kyro Tachikake,False,2023-05-24 03:08:11,Jenova,40,Grade 8 Tincture of Vitality,39729
792,True,5105,3,Ilyesen Valenroix,False,2023-05-24 00:33:49,Cactuar,79,Grade 8 Tincture of Vitality,39729
793,True,30000,3,Ronin Lothbrook,False,2023-05-23 20:35:10,Siren,57,Grade 8 Tincture of Vitality,39729


In [20]:
dexterity_history = pd.json_normalize(tinc_results['entries'][tinc_results['Name'].str.contains("Dexterity")].values[0])
# add the name and itemID of the item to the df
dexterity_history['Name'] = tinc_results['Name'][tinc_results['Name'].str.contains("Dexterity")].values[0]
dexterity_history['itemID'] = tinc_results['itemID'][tinc_results['Name'].str.contains("Dexterity")].values[0]
#convert the timestamp to a date
dexterity_history['timestamp'] = pd.to_datetime(dexterity_history['timestamp'], unit='s')
# save the data to a csv
dexterity_history.to_csv('..\\data\\Dexterity_tincture_market.csv', index=False)
dexterity_history

,hq,pricePerUnit,quantity,buyerName,onMannequin,timestamp,worldName,worldID,Name,itemID
0,True,2862,33,Dark Phenex,False,2024-03-20 17:40:40,Adamantoise,73,Grade 8 Tincture of Dexterity,39728
1,True,2862,33,Dark Phenex,False,2024-03-20 17:40:37,Adamantoise,73,Grade 8 Tincture of Dexterity,39728
2,True,2863,51,Lune Cr'est,False,2024-03-20 16:39:34,Adamantoise,73,Grade 8 Tincture of Dexterity,39728
3,True,2500,33,Dong Senpai,False,2024-03-20 16:09:17,Gilgamesh,63,Grade 8 Tincture of Dexterity,39728
4,True,2961,99,Shadow Erebus,False,2024-03-20 15:53:35,Siren,57,Grade 8 Tincture of Dexterity,39728
...,...,...,...,...,...,...,...,...,...,...
76158,True,14998,33,Shiro Greybourne,False,2023-05-23 14:57:09,Jenova,40,Grade 8 Tincture of Dexterity,39728
76159,True,14998,45,Shiro Greybourne,False,2023-05-23 14:57:08,Jenova,40,Grade 8 Tincture of Dexterity,39728
76160,True,15000,30,Cipher Mss,False,2023-05-23 13:06:54,Jenova,40,Grade 8 Tincture of Dexterity,39728
76161,True,15000,30,Cipher Mss,False,2023-05-23 13:06:52,Jenova,40,Grade 8 Tincture of Dexterity,39728


In [21]:
intelligence_history = pd.json_normalize(tinc_results['entries'][tinc_results['Name'].str.contains("Intelligence")].values[0])
# add the name and itemID of the item to the df
intelligence_history['Name'] = tinc_results['Name'][tinc_results['Name'].str.contains("Intelligence")].values[0]
intelligence_history['itemID'] = tinc_results['itemID'][tinc_results['Name'].str.contains("Intelligence")].values[0]
#convert the timestamp to a date
intelligence_history['timestamp'] = pd.to_datetime(intelligence_history['timestamp'], unit='s')
# save the data to a csv
intelligence_history.to_csv('..\\data\\Intelligence_tincture_market.csv', index=False)
intelligence_history

,hq,pricePerUnit,quantity,buyerName,onMannequin,timestamp,worldName,worldID,Name,itemID
0,True,2823,25,Coconut Melk,False,2024-03-20 16:22:20,Midgardsormr,65,Grade 8 Tincture of Intelligence,39730
1,False,2000,3,Deimos Drackin,False,2024-03-20 15:36:20,Gilgamesh,63,Grade 8 Tincture of Intelligence,39730
2,False,2997,15,Deimos Drackin,False,2024-03-20 15:36:04,Gilgamesh,63,Grade 8 Tincture of Intelligence,39730
3,True,2823,25,Faeron Evergleam,False,2024-03-20 15:13:57,Midgardsormr,65,Grade 8 Tincture of Intelligence,39730
4,True,2737,57,Liri Brightstar,False,2024-03-20 14:14:08,Adamantoise,73,Grade 8 Tincture of Intelligence,39730
...,...,...,...,...,...,...,...,...,...,...
65995,True,15000,10,Vox Veritatis,False,2023-05-23 17:13:01,Jenova,40,Grade 8 Tincture of Intelligence,39730
65996,True,30000,33,Camellia Yagami,False,2023-05-23 15:10:54,Siren,57,Grade 8 Tincture of Intelligence,39730
65997,True,30000,33,Camellia Yagami,False,2023-05-23 15:10:53,Siren,57,Grade 8 Tincture of Intelligence,39730
65998,True,30000,33,Camellia Yagami,False,2023-05-23 15:10:52,Siren,57,Grade 8 Tincture of Intelligence,39730


In [22]:
mind_history = pd.json_normalize(tinc_results['entries'][tinc_results['Name'].str.contains("Mind")].values[0])
# add the name and itemID of the item to the df
mind_history['Name'] = tinc_results['Name'][tinc_results['Name'].str.contains("Mind")].values[0]
mind_history['itemID'] = tinc_results['itemID'][tinc_results['Name'].str.contains("Mind")].values[0]
#convert the timestamp to a date
mind_history['timestamp'] = pd.to_datetime(mind_history['timestamp'], unit='s')
# save the data to a csv
mind_history.to_csv('..\\data\\Mind_tincture_market.csv', index=False)
mind_history

,hq,pricePerUnit,quantity,buyerName,onMannequin,timestamp,worldName,worldID,Name,itemID
0,True,2837,50,Leandros Adastros,False,2024-03-20 11:32:32,Adamantoise,73,Grade 8 Tincture of Mind,39731
1,True,3089,99,Ren Ryder,False,2024-03-20 08:55:08,Faerie,54,Grade 8 Tincture of Mind,39731
2,True,2672,99,Camellia Yagami,False,2024-03-20 07:01:38,Gilgamesh,63,Grade 8 Tincture of Mind,39731
3,True,2672,99,Camellia Yagami,False,2024-03-20 07:01:37,Gilgamesh,63,Grade 8 Tincture of Mind,39731
4,True,2672,99,Camellia Yagami,False,2024-03-20 07:01:36,Gilgamesh,63,Grade 8 Tincture of Mind,39731
...,...,...,...,...,...,...,...,...,...,...
74003,True,13000,27,Halo Nya,False,2023-05-23 13:55:49,Jenova,40,Grade 8 Tincture of Mind,39731
74004,True,40000,3,Atropia Shade,False,2023-05-23 13:28:50,Gilgamesh,63,Grade 8 Tincture of Mind,39731
74005,True,24999,9,Atropia Shade,False,2023-05-23 13:28:49,Gilgamesh,63,Grade 8 Tincture of Mind,39731
74006,True,19000,50,Atropia Shade,False,2023-05-23 13:28:47,Gilgamesh,63,Grade 8 Tincture of Mind,39731


In [23]:
tinctures

,ID,Icon,Name,Url,UrlType,_,_Score
0,39727,/i/020000/020710.png,Grade 8 Tincture of Strength,/Item/39727,Item,item,0
1,39728,/i/020000/020709.png,Grade 8 Tincture of Dexterity,/Item/39728,Item,item,0
2,39729,/i/020000/020707.png,Grade 8 Tincture of Vitality,/Item/39729,Item,item,0
3,39730,/i/020000/020706.png,Grade 8 Tincture of Intelligence,/Item/39730,Item,item,0
4,39731,/i/020000/020708.png,Grade 8 Tincture of Mind,/Item/39731,Item,item,0
